# https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html

https://medium.com/analytics-vidhya/apache-spark-structured-streaming-with-pyspark-b4a054a7947d

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [2]:
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

22/02/26 19:04:23 WARN Utils: Your hostname, LAPTOP-58RVEH10 resolves to a loopback address: 127.0.1.1; using 172.30.89.55 instead (on interface eth0)
22/02/26 19:04:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/26 19:04:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

22/02/26 19:04:35 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [5]:
# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

In [6]:
# Generate running word count
wordCounts = words.groupBy("word").count()

# MAKE SURE nc -lk 9999 is already started on another terminal

In [8]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

22/02/26 19:05:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3a6f56ff-0280-4fc3-bedb-842ac7904c95. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/02/26 19:05:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
|   t|    6|
+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
|fggf|    1|
|   t|    6|
+----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
| fggf|    1|
|saket|    1|
|    t|    6|
+-----+-----+



...

-------------------------------------------
Batch: 9
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|   kopal|    1|
|   kopla|    1|
|    fggf|    1|
|fsdlfksd|    1|
| fdslfks|    2|
|       d|    3|
|   saket|    2|
| fdsslfk|    1|
|       a|    4|
|        |    1|
|       t|    6|
+--------+-----+



-------------------------------------------
Batch: 10
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|   kopal|    1|
|   kopla|    1|
|       k|    1|
|    fggf|    1|
|fsdlfksd|    1|
| fdslfks|    2|
|       d|    5|
|   saket|    2|
| fdsslfk|    1|
|       a|    4|
|        |    1|
|       t|    6|
+--------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/saket/anaconda3/envs/streaming/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/saket/anaconda3/envs/streaming/lib/python3.9/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/saket/anaconda3/envs/streaming/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

22/02/26 19:07:24 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999
